# Problema 3: Optimización de Ti en Estructura Cristalina 3D (NdFe₁₂)
---

## 1. Setup e Imports

In [ ]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print(f"Project root: {project_root}")

In [ ]:
import numpy as np
import plotly.graph_objects as go
import time
print("✓ Imports completados")

In [ ]:
from src.punto3.morse import preparar_morse_params_array
from src.punto3.crystal import (
    get_Nd_positions_fijas,
    get_Fe_positions_all,
    crear_configuracion_inicial,
    compute_total_energy_fast_3d
)
from src.punto3.optimization import simulated_annealing_logarithmic_3d
from src.punto3.optimization.parallel_runs import (
    ejecutar_multiples_runs_logarithmic_3d,
    get_best_run,
    get_run_statistics
)
from src.punto3.analysis import analizar_patron_espacial_3d, interpretar_patron_3d
from src.punto3.visualization import (
    plot_crystal_configuration_3d,
    plot_energy_evolution_3d,
    plot_multiple_runs_comparison_3d
)
print("✓ Imports del proyecto completados")

In [ ]:
morse_params = preparar_morse_params_array()
print(f"Parámetros Morse shape: {morse_params.shape}")
print("✓ Parámetros cargados")

## 2. Estructura Cristalina 3D: Supercelda 2×2×1

Este problema trabaja con la estructura cristalina real de NdFe₁₂ en 3D:
- **16 átomos de Nd (Neodimio)** - tierras raras
- **96 átomos de Fe (Hierro)** - sitios candidatos
- **Objetivo**: Encontrar configuración óptima para **8 átomos de Ti** que sustituyen a 8 Fe

Espacio combinatorial: $\binom{96}{8} \approx 1.86 \times 10^{11}$ configuraciones

In [ ]:
# Obtener posiciones de la estructura cristalina
Nd_positions = get_Nd_positions_fijas()
Fe_all_positions = get_Fe_positions_all()

print(f"Nd positions shape: {Nd_positions.shape}")
print(f"Fe positions shape: {Fe_all_positions.shape}")
print(f"\nTotal de posiciones: {len(Nd_positions) + len(Fe_all_positions)}")

In [ ]:
# Crear configuración inicial aleatoria
all_positions, atom_types, Ti_indices, Fe_indices, _ = crear_configuracion_inicial(seed=69)

print(f"All positions shape: {all_positions.shape}")
print(f"Fe: {np.sum(atom_types == 0)}, Nd: {np.sum(atom_types == 1)}, Ti: {np.sum(atom_types == 2)}")
print(f"Ti indices: {Ti_indices}")

In [ ]:
# Calcular energía inicial
energia_inicial = compute_total_energy_fast_3d(all_positions, atom_types, morse_params)
print(f"Energía inicial: {energia_inicial:.6f}")

## 3. Visualización 3D de la Estructura Inicial

In [ ]:
fig = plot_crystal_configuration_3d(
    all_positions,
    atom_types,
    Nd_positions,
    Ti_indices,
    energia=energia_inicial,
    title="Configuración Inicial - Estructura NdFe₁₂"
)
fig.show()

## 4. Implementación de Simulated Annealing

### Parámetros para Enfriamiento Logarítmico

T(t) = c / log(t + t₀)

Según el Teorema de Hajek (1988):
- c debe ser ≥ Δ (profundidad máxima de barreras de energía)
- t₀ típicamente 2 (para evitar log(0))

In [ ]:
c = 3000  # Constante de enfriamiento
t0 = 2    # Offset temporal
max_iter = 5000000  # Número de iteraciones

print("="*70)
print("PARÁMETROS DE SIMULATED ANNEALING - ENFRIAMIENTO LOGARÍTMICO (3D)")
print("="*70)
print(f"  Esquema:      T(t) = c / log(t + t₀)")
print(f"  Constante c:  {c}")
print(f"  Offset t₀:    {t0}")
print(f"  Iteraciones:  {max_iter:,}")
print(f"\n  Temperatura inicial:  T(0) = {c / np.log(0 + t0):.2f}")
print(f"  Temperatura final:    T({max_iter:,}) = {c / np.log(max_iter + t0):.2f}")
print(f"\n  Espacio de búsqueda:  C(96, 8) ≈ 1.86 × 10¹¹")
print("="*70)

## 5. Ejecución de Múltiples Runs en Paralelo

In [ ]:
N_RUNS = 64

print(f"Ejecutando {N_RUNS} runs en paralelo...")

t_inicio = time.time()
resultados = ejecutar_multiples_runs_logarithmic_3d(
    N_RUNS, 
    c, 
    t0, 
    max_iter, 
    morse_params,
    save_every=10,
    n_jobs=-1, 
    verbose=10
)
t_total = time.time() - t_inicio

print(f"\n✓ Completado en {t_total:.2f}s ({t_total/N_RUNS:.2f}s/run)")

In [ ]:
mejor_run = get_best_run(resultados)
stats = get_run_statistics(resultados)

print(f"Mejor energía: {mejor_run['energia_final']:.6f}")
print(f"Media: {stats['energia_media']:.6f} ± {stats['energia_std']:.6f}")
print(f"Rango: [{stats['energia_min']:.6f}, {stats['energia_max']:.6f}]")

## 6. Análisis de Convergencia

In [ ]:
print("\n" + "="*70)
print("CONVERGENCIA AL ÓPTIMO - MEJOR RUN")
print("="*70)
print(f"Run ID:                     {mejor_run['run_id']}")
print(f"Iteración de convergencia:  {mejor_run['iterations_to_best']:,}/{max_iter:,}")
print(f"Porcentaje del tiempo:      {(mejor_run['iterations_to_best']/max_iter)*100:.1f}%")
print(f"Temperatura en convergencia: T = {c / np.log(mejor_run['iterations_to_best'] + t0):.2f}")
print("="*70)

# Estadísticas de convergencia de TODOS los runs
iter_convergencias = [r['iterations_to_best'] for r in resultados]
print(f"\nESTADÍSTICAS DE CONVERGENCIA (todos los {N_RUNS} runs):")
print(f"  Media:    {np.mean(iter_convergencias):,.0f} iteraciones")
print(f"  Mediana:  {np.median(iter_convergencias):,.0f} iteraciones")
print(f"  Mínimo:   {np.min(iter_convergencias):,} iteraciones")
print(f"  Máximo:   {np.max(iter_convergencias):,} iteraciones")
print(f"  Std:      {np.std(iter_convergencias):,.0f} iteraciones")
print("="*70)

## 7. Visualización de la Configuración Óptima

In [ ]:
# Reconstruir all_positions para el mejor run (las posiciones no cambian, solo atom_types)
atom_types_best = mejor_run['atom_types_best']
Ti_indices_best = mejor_run['Ti_indices_best']

fig = plot_crystal_configuration_3d(
    all_positions,
    atom_types_best,
    Nd_positions,
    Ti_indices_best,
    energia=mejor_run['energia_final'],
    title=f"Configuración Óptima (Run {mejor_run['run_id']})"
)
fig.show()

## 8. Evolución de la Energía

In [ ]:
# Preparar historia del mejor run
history_mejor = {
    'energy': mejor_run['energy_history'],
    'iterations_to_best': mejor_run['iterations_to_best'],
    'temperature': c / np.log(np.arange(len(mejor_run['energy_history'])) * 10 + t0)
}

fig = plot_energy_evolution_3d(history_mejor, highlight_convergence=True, figsize=(16, 7))
fig.show()

print(f"\nEl óptimo se encontró en la iteración {mejor_run['iterations_to_best']:,} de {max_iter:,}")
print(f"({(mejor_run['iterations_to_best']/max_iter)*100:.1f}% del tiempo total)")
print(f"Temperatura en convergencia: T = {c / np.log(mejor_run['iterations_to_best'] + t0):.2f}")

## 9. Comparación de Múltiples Runs

In [ ]:
fig = plot_multiple_runs_comparison_3d(resultados, top_n=20)
fig.show()

## 10. Análisis Espacial de la Configuración Óptima

In [ ]:
patron = analizar_patron_espacial_3d(
    all_positions,
    atom_types_best,
    Ti_indices_best,
    Nd_positions
)

print("\nANÁLISIS ESPACIAL 3D")
print("="*70)
print(f"Distancia Ti-Nd promedio:  {patron['dist_Ti_Nd_promedio']:.3f} Å")
print(f"Distancia Ti-Nd mín:       {patron['dist_Ti_Nd_min']:.3f} Å")
print(f"Distancia Ti-Nd máx:       {patron['dist_Ti_Nd_max']:.3f} Å")
print()
print(f"Distancia Ti-Ti promedio:  {patron['dist_Ti_Ti_promedio']:.3f} Å")
print(f"Distancia Ti-Ti mín:       {patron['dist_Ti_Ti_min']:.3f} Å")
print(f"Distancia Ti-Ti máx:       {patron['dist_Ti_Ti_max']:.3f} Å")
print()
print(f"Clustering score:          {patron['clustering_score']:.3f}")
print(f"Dist. al centro promedio:  {patron['dist_centro_promedio']:.3f} Å")
print("="*70)

In [ ]:
interpretacion = interpretar_patron_3d(patron)
print("\nINTERPRETACIÓN:")
print(interpretacion)

## 11. Posiciones de Ti Optimizadas

In [ ]:
Ti_positions_optimal = all_positions[Ti_indices_best]

print("\nPOSICIONES ÓPTIMAS DE TI (Å):")
print("="*70)
print("Índice  |    X       Y       Z")
print("-" * 70)
for i, (idx, pos) in enumerate(zip(Ti_indices_best, Ti_positions_optimal)):
    print(f"Ti {i+1}    | {pos[0]:7.3f} {pos[1]:7.3f} {pos[2]:7.3f}  (global idx: {idx})")
print("="*70)

## 12. Resumen de Resultados

In [ ]:
print("\n" + "="*70)
print("RESUMEN DE RESULTADOS - PROBLEMA 3")
print("="*70)
print(f"\nESTRUCTURA:")
print(f"  - Átomos totales:        {len(all_positions)}")
print(f"  - Nd (fijos):            {np.sum(atom_types_best == 1)}")
print(f"  - Fe (candidatos):       {np.sum(atom_types_best == 0)}")
print(f"  - Ti (optimizados):      {np.sum(atom_types_best == 2)}")
print(f"\nOPTIMIZACIÓN:")
print(f"  - Espacio de búsqueda:   C(96, 8) ≈ 1.86 × 10¹¹")
print(f"  - Runs ejecutados:       {N_RUNS}")
print(f"  - Iteraciones por run:   {max_iter:,}")
print(f"  - Tiempo total:          {t_total:.1f}s ({t_total/60:.1f} min)")
print(f"\nENERGÍA:")
print(f"  - Energía inicial:       {energia_inicial:.6f}")
print(f"  - Energía óptima:        {mejor_run['energia_final']:.6f}")
print(f"  - Mejora:                {(energia_inicial - mejor_run['energia_final']):.6f}")
print(f"  - Mejora relativa:       {mejor_run['mejora_relativa']*100:.2f}%")
print(f"\nCONVERGENCIA:")
print(f"  - Iteración óptima:      {mejor_run['iterations_to_best']:,} ({(mejor_run['iterations_to_best']/max_iter)*100:.1f}%)")
print(f"  - T en convergencia:     {c / np.log(mejor_run['iterations_to_best'] + t0):.2f}")
print(f"\nDISTRIBUCIÓN ESPACIAL:")
print(f"  - Ti-Nd promedio:        {patron['dist_Ti_Nd_promedio']:.3f} Å")
print(f"  - Ti-Ti promedio:        {patron['dist_Ti_Ti_promedio']:.3f} Å")
print(f"  - Clustering:            {patron['clustering_score']:.3f}")
print("="*70)